In [9]:
import sys
sys.path.append('/cluster/sj1/bb_opt/scripts')

In [10]:
%load_ext autoreload
%autoreload 2

import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import collect_stats as cs

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import collect_stats as cs

In [12]:
exp_folder = '/cluster/sj1/bb_opt/experiments/imdbwiki'
batches = [20, 50, 200]
num_acks = 50

In [13]:
filenames = ['wiki']

In [14]:
def sigmoid(x, exp=np.exp):
  return 1.0 / (1.0 + exp(-x))

In [42]:
arrs = {}

In [162]:
to_read = {
    #'ensemble8/o_max_std_2k_reg_maxvar_g00020510_': [None, 'e8_2k_maxvar_g00020510'],
    #'ensemble8/o_max_std_2k_reg_maxinvar_g00020510_': [None, 'e8_2k_maxinvar_g00020510'],
    #'ensemble8/o_max_std_2k_reg_maxvar_g0005102040_': [None, 'e8_2k_maxvar_g0005102040'],
    'ensemble8/o_max_std_2k_reg_maxvar_g0002051020_': [None, 'e8_2k_maxvar_g0002051020'],
    'ensemble8/o_max_std_2k_reg_maxinvar_g0002051020_': [None, 'e8_2k_maxinvar_g0002051020'],
    'ensemble8/o_max_std_2k_reg_maxinoutvar_g0002051020_': [None, 'e8_2k_maxinoutvar_g0002051020'],
    'ensemble8/o_max_std_2k_reg_inverse_density_maxvar_g0002051020_': [None, 'e8_2k_inverse_maxvar_g0002051020'],
}


In [227]:
for k in to_read:
    arrs[k] = to_read[k]
for experiment in to_read:
    print('reading', experiment)
    arrs[experiment][0] = cs.get_data(exp_folder, experiment, batches, num_samples=100)

reading ensemble8/o_max_std_2k_reg_maxvar_g0002051020_
reading sample 1
reading sample 2
reading sample 3
reading sample 4
reading sample 5
reading sample 6
reading sample 7
reading sample 8
reading sample 9
reading sample 10
reading sample 11
reading sample 12
reading sample 13
reading sample 14
reading sample 15
reading sample 16
reading sample 17
reading sample 18
reading sample 19
reading sample 20
reading sample 21
reading sample 22
reading sample 23
reading sample 24
reading sample 25
reading sample 26
reading sample 27
reading sample 28
reading sample 29
reading sample 30
reading sample 31
reading sample 32
reading sample 33
reading sample 34
reading sample 35
reading sample 36
reading sample 37
reading sample 38
reading sample 39
reading sample 40
reading sample 41
reading sample 42
reading sample 43
reading sample 44
reading sample 45
reading sample 46
reading sample 47
reading sample 48
reading sample 49
reading sample 50
reading sample 51
reading sample 52
reading sample 53


In [148]:
to_eval = [
    #'ensemble8/o_max_std_2k_reg_maxvar_g00020510_',
    #'ensemble8/o_max_std_2k_reg_maxinvar_g00020510_',
    #'ensemble8/o_max_std_2k_reg_maxvar_g0005102040_',
    'ensemble8/o_max_std_2k_reg_maxvar_g0002051020_',
    'ensemble8/o_max_std_2k_reg_maxinvar_g0002051020_',
    'ensemble8/o_max_std_2k_reg_maxinoutvar_g0002051020_',
    'ensemble8/o_max_std_2k_reg_inverse_density_maxvar_g0002051020_',
]

In [20]:
test_fn = lambda bs, prop: lambda x, filename : x[bs + 'test_pred_stats'][prop]
bs_test_rmse_fn = lambda x, filename : x['test_pred_stats']['baseline_rmse']
ood_fn = lambda bs, prop: lambda x, filename : x[bs + 'ood_pred_stats'][prop]
bs_ood_rmse_fn = lambda x, filename : x['test_pred_stats']['baseline_rmse']

In [233]:
dist = "test"
dist = "ood"
prop = "log_prob"
#prop = "rmse"
#prop = "pred_std"
#prop = 'rmse_std_corr'
bs = "zero_gamma_"
#bs = "invar_gamma_"

#fn = [globals()[dist+"_" + prop + "_fn"](""), globals()[dist + "_" + prop + "_fn"](bs)]
fn = [globals()[dist+"_fn"]("", prop), globals()[dist+"_fn"](bs, prop)]
cs.prop_test2(50, 
             filenames,
             fn, 
             arrs, 
             to_eval[1],
             0,
             pval_threshold=1, 
             paired_test=True,
            )

comparing <lambda> <lambda>
wiki 0.4614413412557662 -0.05470928888029929 -0.039198030146576884 1 	 (std: 0.12159119569440985 0.12995023975910744 )
combined pval: nan vs 0.46144
count: 1/1


In [205]:
cs.prop_test(50, 
             filenames,
             ood_fn("", "log_prob"), 
             arrs, 
             [to_eval[0], to_eval[1]],
             0,
             pval_threshold=1, 
             paired_test=False,
            )

comparing e8_2k_maxvar_g0002051020 e8_2k_maxinvar_g0002051020
wiki 0.9941955362201875 -0.050210851436371315 -0.050408425069479054 0 	 (std: 0.13336418604092118 0.1259670073034092 )
combined pval: 0.99420 vs nan
count: 0/1


In [ ]:
cs.plot_data_vs_ack_iter(
    50,
    filenames,
    'avg_seeds',
    'rmse',
    test_rmse_fn,
    20,
    arrs,
    to_eval,
    legend_loc=3,
)

In [ ]:
cs.plot_data_vs_ack_iter(
    20,
    filenames,
    'avg_seeds',
    'log_prob',
    lambda x, filename: x['best_gamma'],
    30,
    arrs,
    to_eval,
    legend_loc=3,
)

In [ ]:
cs.plot_data_vs_ack_iter(
    50,
    filenames,
    'avg_seeds',
    'nll',
    lambda x, filename : x['logging'][1]['best']['nll'],
    20,
    arrs,
    to_eval,
    legend_loc=3,
)

In [ ]:
cs.plot_data_vs_ack_iter(
    200,
    filenames,
    'avg_seeds',
    'rmse',
    test_rmse_fn,
    20,
    arrs,
    to_eval,
    legend_loc=3,
)